In [1]:
from Milky_Way_Satellites_CDM_Model import gen_subhalo_pop
from MWS_real_data_distribution import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
# Generic properties from matplotlib
rcParams['font.family'] = 'serif'
rcParams['text.usetex'] = True

In [2]:
# @title Import Python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import scipy
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader, RandomNodeLoader
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import (
    MessagePassing, GCNConv, SAGEConv, EdgeConv,
)
from torch_cluster import radius_graph
from tqdm import tqdm


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [21]:
Mr_real = Mr_real_data
sigma_real = sigma_star_real_data
r_12_real = Halo_r12_real_data

In [18]:
filename_mock = '../mock_data/Milky_Way_Satellites_CDM_Mock_Data.dat'

Milky_Way_Mock_Data = np.genfromtxt(filename_mock).T

Mr_mock = Milky_Way_Mock_Data[7]
sigma_star_mock = Milky_Way_Mock_Data[6]
r_12_mock = Milky_Way_Mock_Data[4]

In [26]:
# a smaller subset of the full data
train_mask = np.array([Mr_mock, r_12_mock, sigma_star_mock])
valid_mask = np.array([Mr_real, r_12_real, sigma_real])

# note -- this is the equivalent of `np.argwhere()`
train_indices = torch.tensor(train_mask).nonzero(as_tuple=True)[0]
valid_indices = torch.tensor(valid_mask).nonzero(as_tuple=True)[0]

assert (set(train_indices) & set(valid_indices)) == set()

In [58]:
# train-test split
X_train = np.zeros(train_mask[0].size)
for i in range(train_mask[0].size):
    X_train[i] = np.array([train_mask[0][i], train_mask[1][i]])
y_train = np.array([train_mask[2]])
X_valid = np.array([valid_mask[0], valid_mask[1]])
y_valid = np.array([valid_mask[2]])

ValueError: setting an array element with a sequence.

249.64090592331303

In [48]:
# random forest <3
rf = RandomForestRegressor(n_estimators=100, random_state=None)
rf.fit(X_train.ravel(), y_train)
p_valid = rf.predict(X_valid)

ValueError: Expected 2D array, got 1D array instead:
array=[ 6.351169   6.35889    6.3435283 ... 16.        16.        16.       ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.